# Testing & Iteration

The development of the AI-driven personal assistant involves a rigorous process of testing and iteration to ensure functionality and reliability.

## Initial Implementation: Speech Recognition from Audio File

Initially, I implemented basic functionality to transcribe audio to text. This served as the foundation for speech recognition. The code here allowed recognition of speech from a pre-recorded audio file.

*Comment: Code to recognize speech from an audio file.*
**Code**:
```python
def recognize_from_audio(audio_file_path):
    # Set up the speech configuration with your subscription info and region
    speech_config = speechsdk.SpeechConfig(subscription=API_KEY, region=REGION)
    
    # Specify the audio file to recognize
    audio_config = speechsdk.audio.AudioConfig(filename=audio_file_path)

    # Create a speech recognizer using the given settings
    recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    # Start speech recognition and get the result
    result = recognizer.recognize_once()

    # Check the result
    if result.reason == speechsdk.ResultReason.RecognizedSpeech:
        print(f"Recognized: {result.text}")
    elif result.reason == speechsdk.ResultReason.NoMatch:
        print("No speech could be recognized")
    elif result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = result.cancellation_details
        print(f"Speech Recognition canceled: {cancellation_details.reason}")
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            print(f"Error details: {cancellation_details.error_details}")

# Replace with the path to the audio file you want to transcribe
audio_file_path = "KSA2.wav"
```


## Enhancing Real-Time Voice Input

Following the initial success, I modified the implementation to support real-time voice input via the microphone. This enhancement allowed dynamic interaction between users and the assistant.



*Comment: Code to enable real-time speech recognition via microphone.*

**Code**:
```python
def recognize_speech_continuously():
    # Create a speech configuration with your subscription info and region
    speech_config = speechsdk.SpeechConfig(subscription=API_KEY, region=REGION)
    
    # Create a speech recognizer using the default microphone
    audio_config = speechsdk.audio.AudioConfig(use_default_microphone=True)
    recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    print("Listening... Speak into your microphone.")

    # Start continuous recognition
    def recognized_handler(evt):
        if evt.result.reason == speechsdk.ResultReason.RecognizedSpeech:
            print(f"Recognized: {evt.result.text}")
            # Here you can process the recognized text (e.g., send it to your agent for a response)
        elif evt.result.reason == speechsdk.ResultReason.NoMatch:
            print("No speech could be recognized.")
        elif evt.result.reason == speechsdk.ResultReason.Canceled:
            print(f"Recognition canceled: {evt.result.cancellation_details.reason}")

    # Use the `add_event` method to register the event handler
    recognizer.recognized.connect(recognized_handler)

    # Start the continuous recognition
    recognizer.start_continuous_recognition()

    # Keep the application running until you stop it (e.g., use a keyboard interrupt)
    try:
        while True:
            pass  # Keep the script running
    except KeyboardInterrupt:
        print("Stopping recognition...")
        recognizer.stop_continuous_recognition()
```

## Managing Two-Way Conversation

To enable effective two-way communication, I introduced a `call_state` variable to manage the assistant's current state, either "LISTENING" or "SPEAKING." This addition controlled when the assistant listened for user input and when it generated responses. 



*Comment: Code to manage assistant's state for two-way conversation .*

**Code**:
```python


def recognize_speech_continuously():
    global call_state,recognizer
    speech_config = speechsdk.SpeechConfig(subscription=API_KEY, region=REGION)
    audio_config = speechsdk.audio.AudioConfig(use_default_microphone=True)
    recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)
    
    
    print("[DEBUG] ASR started and is now listening...")
    initiate_conversation_with_llm()
    print("Listening... Speak into your microphone.")

    def recognized_handler(evt):
        global call_state,recognizer
        if call_state != "LISTENING":
            print("[DEBUG] ASR ignored input since call_state is not LISTENING.")
            return

        # Process recognized speech
        print("LISTINGIGN AGAIN")
        print(f"speechsdk.ResultReason.RecognizedSpeech :{speechsdk.ResultReason.RecognizedSpeech}")
        if evt.result.reason == speechsdk.ResultReason.RecognizedSpeech:
            recognized_text = evt.result.text.strip()
            if recognized_text:
                print(f"[DEBUG] ASR recognized text: {recognized_text}")
                call_state = "SPEAKING"  # Switch to speaking state
                recognizer.stop_continuous_recognition()  # Stop ASR during response processing
                print("[DEBUG] ASR stopped for LLM response processing.")
                
                # Process the recognized text with process_chunk
                llm_response, function_name, function_arguments, function_id, first_chunk_time, error_occurred = process_chunk(recognized_text)
                
                # Handle the LLM response
                if llm_response:
                    messages.append({"role": "assistant", "content": llm_response})
                elif function_name:
                    messages.append(
                            {
                                "role": "assistant",
                                "content": None,
                                "tool_calls": [
                                    {
                                        "id": function_id,
                                        "function": {"name": function_name, "arguments": function_arguments},
                                        "type": "function",
                                    }
                                ],
                            }
                        )
                    print(f"[DEBUG] Calling function '{function_name}' with arguments: {function_arguments}")
                                # Call the WhatsApp sending function
                    function_returns = send_message_to_whatsapp(function_arguments)
                    messages.append(
                        {
                            "tool_call_id": function_id,
                            "role": "tool",
                            "name": function_name,
                            "content": function_returns,
                        }
                    )
                    llm_response, _, _, _, _, _ = process_chunk(None)
                    print(f"LLM Response after funtion call:- {llm_response}")
                    messages.append({"role":"assistant","content":llm_response})
                    call_state = "LISTENING"
                recognizer.start_continuous_recognition()    
        elif evt.result.reason == speechsdk.ResultReason.NoMatch:
            print("[DEBUG] No speech could be recognized.")
        elif evt.result.reason == speechsdk.ResultReason.Canceled:
            print(f"[DEBUG] Recognition canceled: {evt.result.cancellation_details.reason}")

    recognizer.recognized.connect(recognized_handler)
    # recognizer.start_continuous_recognition()
    # Start ASR only if it's in the correct state
    if call_state == "LISTENING":
        recognizer.start_continuous_recognition()
        print("[DEBUG] ASR started and is now listening...")
    print("[DEBUG] ASR started and is now listening...")
    
    try:
        while True:
            if call_state == "STOP":
                recognizer.stop_continuous_recognition()
                print("[DEBUG] ASR now is STOPPOING")
                break 
    except KeyboardInterrupt:
        print(f"traceback in recognize_speech_continuously : {traceback.format_exc()}")
        recognizer.stop_continuous_recognition()
```
## Iterative Refinement

This is the final refined code for the Automatic Speech Recognition (ASR) system, which enhances readability and passes the recognized text to the Language Model (LLM) for further processing.

**Code**:
```python

#-------------------------------------------------------------------------------------------------------
def initialize_speech_recognizer():
    """Initialize the Azure speech recognizer."""
    speech_config = speechsdk.SpeechConfig(subscription=AZURE_API_KEY, region=AZURE_REGION)
    audio_config = speechsdk.audio.AudioConfig(use_default_microphone=True)
    recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)
    return recognizer


#-------------------------------------------------------------------------------------------------------
def start_recognition_if_listening(recognizer):
    """Start continuous recognition if the call state is LISTENING."""
    global call_state  # Access the global call_state variable
    if call_state == "LISTENING":
        recognizer.start_continuous_recognition()
        print("[DEBUG] ASR started and is now listening...")
    else:
        print("[DEBUG] ASR is not in the LISTENING state.")

#-------------------------------------------------------------------------------------------------------
def process_recognized_text(recognized_text, recognizer):
    """Process the recognized text and interact with the LLM."""
    global call_state  # Access the global call_state variable
    change_call_state("SPEAKING")
    recognizer.stop_continuous_recognition()  # Stop ASR during response processing
    print("[DEBUG] ASR stopped for LLM response processing.")


#-------------------------------------------------------------------------------------------------------
def log_llm_response(llm_response, function_name, function_arguments, function_id, first_chunk_time, error_occurred):
    """Log details of the LLM response."""
    print(f"[DEBUG] LLM Response: {llm_response}")
    print(f"[DEBUG] Function Name: {function_name}")
    print(f"[DEBUG] Function Arguments: {function_arguments}")
    print(f"[DEBUG] Function ID: {function_id}")
    print(f"[DEBUG] First Chunk Time: {first_chunk_time} ms")
    print(f"[DEBUG] Error Occurred: {error_occurred}")


#-------------------------------------------------------------------------------------------------------
def handle_llm_response(llm_response, function_name, function_arguments, function_id):
    """Handle the LLM response appropriately."""
    global call_state  # Access the global call_state variable
    if llm_response:
        messages.append({"role": "assistant", "content": llm_response})
    elif function_name:
        append_asst_msg(messages=messages, function_id=function_id, function_name=function_name,
                        function_args=function_arguments)
        print(f"[DEBUG] Calling function '{function_name}' with arguments: {function_arguments}")
        # Call the WhatsApp sending function
        function_returns = send_message_to_whatsapp(function_arguments)
        append_tool_call_message(messages=messages, function_id=function_id, function_name=function_name,
                                 function_returns=function_returns)
        llm_response, _, _, _, _, _ = process_chunk(None, client, messages, tools)
        print(f"LLM Response after function call: {llm_response}")
        messages.append({"role": "assistant", "content": llm_response})
    
    change_call_state("LISTENING")


#-------------------------------------------------------------------------------------------------------
def handle_recognition_result(evt, recognizer):
    """Handle the recognition result from the speech recognizer."""
    global call_state  # Access the global call_state variable

    if call_state != "LISTENING":
        print("[DEBUG] ASR ignored input since call_state is not LISTENING.")
        return

    print("[DEBUG] LISTENING AGAIN")
    if evt.result.reason == speechsdk.ResultReason.RecognizedSpeech:
        recognized_text = evt.result.text.strip()
        if recognized_text:
            print(f"[DEBUG] ASR recognized text: {recognized_text}")
            process_recognized_text(recognized_text, recognizer)
    elif evt.result.reason == speechsdk.ResultReason.NoMatch:
        print("[DEBUG] No speech could be recognized.")
    elif evt.result.reason == speechsdk.ResultReason.Canceled:
        print(f"[DEBUG] Recognition canceled: {evt.result.cancellation_details.reason}")



    # Process the recognized text with process_chunk
    llm_response, function_name, function_arguments, function_id, first_chunk_time, error_occurred = process_chunk(recognized_text, client, messages, tools)
    
    # Log the results
    log_llm_response(llm_response, function_name, function_arguments, function_id, first_chunk_time, error_occurred)

    # Handle the LLM response
    handle_llm_response(llm_response, function_name, function_arguments, function_id)
    recognizer.start_continuous_recognition()


#-------------------------------------------------------------------------------------------------------
def recognize_speech_continuously():
    """Continuously recognize speech using Azure Speech Service.""" 
    call_state = get_call_state()  
    
    recognizer = initialize_speech_recognizer()
    change_recognizer(recognizer)  # Ensure the recognizer is set globally
    print("[DEBUG] ASR started and is now listening...")
    print("[DEBUG] ASR initialized. Waiting to start recognition...")
    
    initiate_conversation_with_llm()
    print("Listening... Speak into your microphone.")
    
    recognizer.recognized.connect(lambda evt: handle_recognition_result(evt, recognizer))
    
    # Start ASR only if it's in the correct state
    start_recognition_if_listening(recognizer)
    
    try:
        while True:
            call_state = get_call_state()
            if call_state == "STOP":
                recognizer.stop_continuous_recognition()
                print("[DEBUG] ASR is now STOPPING")
                break 
    except KeyboardInterrupt:
        print("[DEBUG] Stopping recognition...")
        print(f"traceback in recognize_speech_continuously: {traceback.format_exc()}")
        recognizer.stop_continuous_recognition()

```

**Through this iterative process of coding, testing, and refining, I was able to enhance the Azure ASR's capabilities, ensuring a smooth and engaging user experience. Each modification was guided by testing outcomes, allowing me to build a robust solution that effectively meets user needs.**



# Development of System Prompt through Testing and Iteration

## 1. Initial Version (Basic Level)
This is a simple outline without much detail.

```plaintext
Your role is to be a personal assistant for Mr. Ravi Ranjan. Manage his calls when he is not available.

1. Greet the caller.
2. Ask for their name and message.
3. Summarize the message.
4. Send the summary to Mr. Ravi Ranjan.
5. Be professional and polite.
```

## Sample Outputs
**Caller**: Hi, is Mr. Ravi Ranjan available?  
**AI Assistant**: Hello, I’m the personal assistant for Mr. Ravi Ranjan. He is currently unavailable.  

**Caller**: I’m Rahul. I need to discuss an urgent business matter.  
**AI Assistant**: Got it, Rahul. I’ll relay that message to Mr. Ranjan.

## 2. Enhanced Version
Added more detail and clarified tasks. 

```plaintext
You are Mr. Ravi Ranjan’s personal assistant. Your job is to handle calls when he can't.

1. Greet the caller and let them know he is unavailable.
2. Ask for the caller’s name and the reason for their call.
3. Encourage them to provide details about their message.
4. Summarize what they say, capturing important points.
5. Confirm the message with the caller before sending it.
6. Send the summary to Mr. Ravi Ranjan via WhatsApp.
7. Maintain a professional and respectful tone.

```
## Sample Outputs
**Caller**: Hi, is Mr. Ravi Ranjan available?  
**AI Assistant**: Hello! I’m Mr. Ravi Ranjan’s personal assistant. He’s currently unavailable.  

**Caller**: I’m Rahul. I need to discuss an urgent business matter.  
**AI Assistant**: Thank you, Rahul. Could you please explain the reason for your call and share any important details?  

**Caller**: I need to finalize the contract by the end of the week.  
**AI Assistant**: Thank you for sharing, Rahul. To confirm, you mentioned that you need to finalize the contract by the end of the week. Is that correct?  

**Caller**: Yes, that’s correct.  
**AI Assistant**: Great! I’ll make sure Mr. Ranjan receives this message.


## 3. Structured Version
Introduced clearer structure and formatting for readability.

```plaintext
As Mr. Ravi Ranjan’s personal assistant, you will manage his calls. Your tasks include:

1. **Greeting the Caller**: Inform them that Mr. Ranjan is unavailable.
2. **Capturing Information**:
   - Request the caller's name and reason for their call.
   - Encourage detailed messages (e.g., “Please provide any important details.”).
3. **Message Confirmation**:
   - Summarize the caller’s message.
   - Confirm the summary with the caller.
4. **Sending the Summary**: 
   - Relay the message to Mr. Ranjan via WhatsApp.
5. **Professional Tone**: 
   - Always be respectful and professional.

```

## Sample Outputs
**Caller**: Hi, is Mr. Ravi Ranjan available?  
**AI Assistant**: Hello, I’m LinkDesk, Mr. Ravi Ranjan’s personal assistant. He is currently unavailable.  

**Caller**: I’m Rahul. I need to discuss an urgent business matter.  
**AI Assistant**: Thank you, Rahul. Could you please provide me with the details of your message?  

**Caller**: I need to finalize the contract by the end of the week.  
**AI Assistant**: Thank you for that information. To confirm, you mentioned needing to finalize the contract by the end of the week. Is that correct?  

**Caller**: Yes, that’s right.  
**AI Assistant**: Thank you! I will send this summary to Mr. Ranjan now.



## 4. Detailed Version
Added specific language for guidance and behavior.

```plaintext
You are LinkDesk, the smart personal assistant for Mr. Ravi Ranjan. Your mission is to manage his calls when he is unavailable:

1. **Greeting the Caller**: 
   - Introduce yourself as Mr. Ranjan’s personal assistant and inform the caller of his unavailability.

2. **Capturing the Caller’s Message**:
   - Ask for their name and the purpose of their call.
   - Encourage detailed messages (e.g., “Could you please explain the reason for your call and share any important details?”).
   - Accurately summarize key points and urgent matters.

3. **Confirming the Message**:
   - Confirm the captured message (e.g., “Thank you for sharing. To confirm, you mentioned [insert summary]. Is that correct?”).

4. **Sending the Summary**:
   - Prepare a clear summary of the caller’s message and send it to Mr. Ranjan via WhatsApp.
   - Inform the caller that their message has been recorded.

5. **Maintaining Professionalism**: 
   - Use a professional and humble tone, ensuring the caller feels valued.

```

## Sample Outputs
**Caller**: Hi, is Mr. Ravi Ranjan available?  
**AI Assistant**: Hello! I’m LinkDesk, Mr. Ravi Ranjan’s personal assistant. He is currently unavailable.  

**Caller**: I’m Rahul. I need to discuss an urgent business matter.  
**AI Assistant**: Thank you, Rahul. May I ask for your name and the purpose of your call?  

**Caller**: My name is Rahul, and I need to finalize the contract by the end of the week.  
**AI Assistant**: Thank you for sharing, Rahul. To confirm, you mentioned that you need to finalize the contract by the end of the week. Is that correct?  

**Caller**: Yes, that’s correct.  
**AI Assistant**: I will prepare a summary of your message and send it to Mr. Ranjan via WhatsApp. Thank you for your patience.


## 5.Better Version
all refinements and the final structure

```plaintext
Your role is to serve as LinkDesk, the smart personal assistant for {recipient_name}, responsible for managing his calls when he's unavailable. Here's your mission:

1. **Greeting the Caller**: Introduce yourself as {recipient_name}’s  personal assistant and inform the caller that he is currently unavailable.

2. **Capturing the Caller’s Message**:
   - Ask the caller for their name and the purpose of their call.
   - Encourage them to share a detailed message, guiding them if needed (e.g., “Could you please explain the reason for your call and share any important details?”).
   - Summarize the caller’s message, ensuring key points and any urgent matters are noted accurately.

3. **Confirming the Message**:
   - After the caller finishes, confirm the captured message. For example, say, “Thank you for sharing. To confirm, you mentioned [insert summary]. Is that correct?”

4. **Sending the Summary**:
   - Prepare a concise and clear summary of the caller’s message, including their name and purpose of the call.
   - Use the send_message_to_whatsapp tool to relay this summary promptly to {recipient_name}  via his preferred communication channel (e.g., WhatsApp).
   - Inform the caller politely that their message has been recorded and shared with {recipient_name}.

5. **Maintaining Professionalism with Humility**:
   - Use a professional yet humble tone in all interactions, reflecting positively on {recipient_name}.
   - Ensure the caller feels respected and valued, regardless of the nature of their inquiry.

Your goal is to ensure seamless communication between callers and {recipient_name}, capturing important messages accurately and delivering them efficiently.
```

## Sample Outputs
**Caller**: Hi, is Mr. Ravi Ranjan available?  
**AI Assistant**: Hello! I’m LinkDesk, Mr. Ravi Ranjan’s personal assistant. He is currently unavailable.  

**Caller**: I’m Rahul. I need to discuss an urgent business matter.  
**AI Assistant**: Thank you, Rahul. Could you please explain the reason for your call and share any important details?  

**Caller**: I need to finalize the contract by the end of the week.  
**AI Assistant**: Thank you for sharing. To confirm, you mentioned that you need to finalize the contract by the end of the week. Is that correct?  

**Caller**: Yes, that’s right.  
**AI Assistant**: Great! I will prepare a concise summary of your message, including your name and the purpose of your call. I’ll send this to Mr. Ranjan via WhatsApp right away. Your message has been recorded and will be shared with him.



# Testing and Iteration for the LLM Module



## Initial Implementation
Initially, I implemented a function to process user requests and interact with the LLM. The goal was to handle basic user inputs and provide corresponding responses efficiently.


**Code**:
```python
def process_chunk(chunk=None):
    if chunk is None:
        print("[ERROR] Empty chunk!")
        return ""
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": chunk}
            ]
        )
        description = response.choices[0].message.content
        print("[DEBUG] Processed chunk successfully.")  
        return description
    except Exception as e:
        print(f"[ERROR] An error occurred while processing the chunk: {e}")
        return ""

```
**Observation:**  
The initial implementation worked well with smaller inputs. However, processing larger requests led to significant increases in response times.

---

## Enhancing Functionality with Tools
To improve the function's capabilities, I added support for external tools. This allowed the LLM to utilize additional resources for generating responses.

**Code**:
```python
def process_chunk(chunk=None):
    if chunk is None:
        print("[ERROR] Empty chunk!")
        return ""
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": chunk}
            ],
            tools=tools
        )
        description = response.choices[0].message.content
        print("[DEBUG] Processed chunk successfully.")  
        return description
    except Exception as e:
        print(f"[ERROR] An error occurred while processing the chunk: {e}")
        return ""

```

**Testing Outcome:**  
Despite the enhancement, response times for larger inputs were still slower than desired. This prompted further exploration to optimize the performance.

---

## Implementing Streaming
To address performance concerns, I introduced streaming. This modification allowed the function to receive responses incrementally, improving delivery time for the initial chunks.



**Code**:
```python
def process_chunk(request):
    try:
        # Append user message to the messages array
        if request:
            messages.append({"role": "user", "content": request})

        # Create the chat completion request with streaming
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=tools,
            stream=True
        )

        # Process the streaming response
        for chunk in response:
            print(f"chunk: {chunk}")
    except Exception as e:
        print(f"error occur :{e}") 

```

**Output**:
```python
chunk: ChatCompletionChunk(id='chatcmpl-AebdfzrVJ6aGlaEgPDkjdTyyGuZAG', choices=[Choice(delta=ChoiceDelta(content='', function_call=None, refusal=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1734241167, model='gpt-4o-mini-2024-07-18', object='chat.completion.chunk', service_tier=None, system_fingerprint='fp_6fc10e10eb', usage=None)
chunk: ChatCompletionChunk(id='chatcmpl-AebdfzrVJ6aGlaEgPDkjdTyyGuZAG', choices=[Choice(delta=ChoiceDelta(content='Hello', function_call=None, refusal=None, role=None, tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1734241167, model='gpt-4o-mini-2024-07-18', object='chat.completion.chunk', service_tier=None, system_fingerprint='fp_6fc10e10eb', usage=None)
```

**Observations:**  
Incremental responses were successfully received, but the processing logic needed refinement to manage these chunks effectively.

---

## Refining Chunk Processing Logic
I developed logic to extract and process streamed chunks, implementing punctuation-based splitting to construct coherent outputs.


**Code**:
```python
def process_chunk(request):
    try:
        # Append user message to the messages array
        if request:
            messages.append({"role": "user", "content": request})

        # Create the chat completion request with streaming
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=tools,
            stream=True
        )

        # Initialize variables for processing the response
        complete_string = ""
        continious_string = ""
        chunk_timestamp = time.time()

        # Process the streaming response
        for chunk in response:
            print(f"chunk: {chunk}")
            current_gpt_chunk = chunk.choices[0].delta.content if chunk.choices else ""

            if current_gpt_chunk and chunk.choices and chunk.choices[0].delta:
                # Determine break punctuation
                break_punctuation = [',', '!', '.', '?']
                
                if any(punc in current_gpt_chunk for punc in break_punctuation):
                    continious_string += current_gpt_chunk
                    complete_string += current_gpt_chunk

                    # Print the chunk for debugging
                    print("chunk:->", continious_string)
                    
                    continious_string = ""
                else:
                    continious_string += current_gpt_chunk

        return complete_string

    except Exception as e:
        print(f"[ERROR] An error occurred while processing the chunk: {e}")
        return None

```

**Key Changes:**
1. Introduced variables to manage ongoing and complete strings.
2. Used break punctuation (e.g., `,`, `.`, `!`, `?`) to split and assemble meaningful chunks.

**Output**:
```python
Enter your query: hi how you can help me
[DEBUG] Time taken to generate 1st chunk: 28.403043746948242 ms
chunk:-> Hello!
chunk:-> I am Mr.
chunk:-> Ravi Ranjan’s assistant.
chunk:-> How can I assist you today?
chunk:-> Unfortunately, Mr.
chunk:-> Ranjan is currently unavailable, but I can take your message and ensure he receives it.
chunk:->  Please let me know your name and the purpose of your call!
[DEBUG] LLM Response: Hello! I am Mr. Ravi Ranjan’s assistant. How can I assist you today? Unfortunately, Mr. Ranjan is currently unavailable, but I can take your message and ensure he receives it. Please let me know your name and the purpose of your call!
```

**Testing Outcome:**  
The function handled streaming more effectively, but challenges persisted in managing complex responses with function calls.

---

## Final Adjustments
To handle function calls in streamed responses, I refined the logic further. The final implementation extracts:
1. `function_name` for identifying the called function.
2. `function_arguments` for capturing the arguments passed to the function.

<!-- Placeholder for final refined function with function call handling -->
*(Insert function here if needed to handle function calls in streaming responses.)*
**Code**:
```python
def process_chunk(request):
    try:
        # Append user message to the messages array
        if request:
            messages.append({"role": "user", "content": request})

        # Create the chat completion request with streaming
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=tools,
            stream=True
        )

        # Initialize variables for processing the response
        complete_string = ""
        continious_string = ""
        chunk_timestamp = time.time()

        # Process the streaming response
        for chunk in response:
            print(f"chunk: {chunk}")
            current_gpt_chunk = chunk.choices[0].delta.content if chunk.choices else ""

            if current_gpt_chunk and chunk.choices and chunk.choices[0].delta:
                # Determine break punctuation
                break_punctuation = [',', '!', '.', '?']
                
                if any(punc in current_gpt_chunk for punc in break_punctuation):
                    continious_string += current_gpt_chunk
                    complete_string += current_gpt_chunk

                    # Print the chunk for debugging
                    print("chunk:->", continious_string)
                    
                    continious_string = ""
                else:
                    continious_string += current_gpt_chunk
            if chunk.choices and chunk.choices[0].delta.tool_calls:
                function_name = chunk.choices[0].delta.tool_calls[0].function.name
                function_arguments = chunk.choices[0].delta.tool_calls[0].function.arguments
                print(f"function_name: {function_name}")
                print(f"function_arguments: {function_arguments}")        

        return complete_string

    except Exception as e:
        print(f"[ERROR] An error occurred while processing the chunk: {e}")
        return None

```

**Key Refinements:**
- Added handling for `tool_calls` in the streamed responses.
- Improved error management for edge cases.

---

## Results and Future Work
**Results:**  
- Streaming implementation successfully reduced initial response times.
- The refined logic handled function calls and complex responses effectively.

**Future Work:**
- Optimize response assembly for higher performance.
- Explore further improvements in tool integration to enhance LLM interactions.
